In [15]:
import pandas as pd
import sqlite3

excel_file = "data.xlsx"  
table_name = "my_table"     
db_file    = "SchneiderDatabase.db"  

try:
    df = pd.read_excel(excel_file)  

except FileNotFoundError:
    print(f"Error: Excel file '{excel_file}' not found.")
    exit()  


conn = sqlite3.connect(db_file)

df.to_sql(table_name, conn, if_exists="replace", index=False)  


cursor = conn.cursor()
cursor.execute(f"SELECT * FROM {table_name} LIMIT 5") 
rows = cursor.fetchall()
print("First 5 rows from the database:")
for row in rows:
    print(row)

conn.close()


First 5 rows from the database:
('2K3316/022ST', '合闸线圈(DC220V)', 'FEP', 212, 'PKK', 'PKK', 'Order specific', 'Order specific', '2100', 8.0, 0, 48.86, 0, 0, 0.0, 0.0)
('46K133/001', 'FEP手动储能杆', 'FEP', 212, 'PKK', 'PKK', 'Order specific', 'Order specific', '2100', 1.0, 0, 0.0, 0, 0, 0.0, 0.0)
('AGS000440-01', 'BEARING', 'XA', 202, 'AK4', 'AK4', 'Order specific', 'Order specific', '2100', 10.0, 0, 238.0, 0, 0, 100.0, 0.0)
('AGS000777-01', 'COMPLETE LEVER', 'SM', 212, 'AK4', 'AK4', 'SM', 'kanban', '2600', 116.0, 250, 18.0, 0, 50, 1259.0, 450.0)
('AGS0011K4-01', 'DISK', 'SM', 202, 'AK4', 'AK4', 'SM', 'kanban', '2600', 155.0, 60, 4.51, 0, 30, 875.0, 67.64999999999999)


In [16]:
def get_table_attributes(db_file, table_name):
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        cursor.execute(f"PRAGMA table_info({table_name})")  
        columns = [row[1] for row in cursor.fetchall()]  


        conn.close()
        return columns

    

attributes = get_table_attributes(db_file, table_name)

if attributes:
    print(f"Attributes (columns) of table '{table_name}':")
    for attribute in attributes:
        print(f"- {attribute}")

Attributes (columns) of table 'my_table':
- Part Number
- Description
- Part sub commodity
- PUR Grp
- Mini Business
- Product
- Order specific part
- Procurement mode
- Location
- Stock qty
- Pending orders
- Part standard price
- Safety Stock
- Batch size
- Yearly consumption
- Batch size impact


In [17]:
def calculate_stock_value(db_file, table_name):

 
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        query = f"""
            SELECT
                `Part Number`,
                `Part standard price`,
                `Stock qty`,
                `Part standard price` * `Stock qty` AS StockValue
            FROM
                `{table_name}`
        """ 

        cursor.execute(query)
        rows = cursor.fetchall()

        print("Stock Value Calculation:")
        print("-" * 30)
        for row in rows:
            part_number = row[0]  
            price = row[1]
            quantity = row[2]
            stock_value = row[3]  
            print(f"Part Number: {part_number}, Price: {price}, Quantity: {quantity}, Stock Value: {stock_value:.2f}")  

        conn.close()


calculate_stock_value(db_file, table_name)

Stock Value Calculation:
------------------------------
Part Number: 2K3316/022ST, Price: 48.86, Quantity: 8.0, Stock Value: 390.88
Part Number: 46K133/001, Price: 0.0, Quantity: 1.0, Stock Value: 0.00
Part Number: AGS000440-01, Price: 238.0, Quantity: 10.0, Stock Value: 2380.00
Part Number: AGS000777-01, Price: 18.0, Quantity: 116.0, Stock Value: 2088.00
Part Number: AGS0011K4-01, Price: 4.51, Quantity: 155.0, Stock Value: 699.05
Part Number: AGS00165K-01, Price: 2.5, Quantity: 311.0, Stock Value: 777.50
Part Number: AGS001665-01, Price: 1.14, Quantity: 20.0, Stock Value: 22.80
Part Number: AGS0017K0-01, Price: 20.0, Quantity: 1.0, Stock Value: 20.00
Part Number: AGS002432-01, Price: 0.7, Quantity: 2460.0, Stock Value: 1722.00
Part Number: AGS00243K-01, Price: 5.54, Quantity: 10.0, Stock Value: 55.40
Part Number: AGS002439-01, Price: 1.17, Quantity: 229.0, Stock Value: 267.93
Part Number: AGS002751-01, Price: 201.66, Quantity: 264.0, Stock Value: 53238.24
Part Number: AGS44K71K-01, Pr

In [18]:
def add_stock_value_column(db_file, table_name):
    """
    Adds a 'Stock Value' column to the specified table in the SQLite database
    and calculates the stock value for each row, storing the result in the new column.

    Args:
        db_file (str): Path to the SQLite database file.
        table_name (str): Name of the table.
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    cursor.execute(f"ALTER TABLE `{table_name}` ADD COLUMN `Stock Value` REAL")  

    
    update_query = f"""
        UPDATE `{table_name}`
        SET `Stock Value` = `Part standard price` * `Stock qty`
    """
    cursor.execute(update_query)
    conn.commit()  
    conn.close()




add_stock_value_column(db_file, table_name)

In [19]:

def add_days_column(db_file, table_name):

    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Check if 'Days' column already exists
        cursor.execute(f"PRAGMA table_info(`{table_name}`)")
        columns = [row[1] for row in cursor.fetchall()]
        if 'Days' in columns:
            print(f"Column 'Days' already exists in table '{table_name}'.")
        else:
            # Add the 'Days' column (REAL for potential decimal values)
            cursor.execute(f"ALTER TABLE `{table_name}` ADD COLUMN `Days` REAL")
            print(f"Added 'Days' column to table '{table_name}'.")

        # Update the 'Days' column based on the logic
        update_query = f"""
            UPDATE `{table_name}`
            SET `Days` =
                CASE
                    WHEN `Yearly consumption` = 0 THEN 0
                    ELSE ROUND(CAST(CAST(`Stock qty` AS REAL) / `Yearly consumption` * 365 AS REAL),0)
                END
        """
        cursor.execute(update_query)
        conn.commit()
        print(f"Calculated and updated 'Days' for all rows in table '{table_name}'.")

        conn.close()

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")




add_days_column(db_file, table_name)

Added 'Days' column to table 'my_table'.
Calculated and updated 'Days' for all rows in table 'my_table'.


In [20]:
def add_stock_value_column(db_file, table_name):
    """
    Adds a 'Stock Value' column to the specified table in the SQLite database
    and calculates the stock value for each row, storing the result in the new column.

    Args:
        db_file (str): Path to the SQLite database file.
        table_name (str): Name of the table.
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()


    cursor.execute(f"PRAGMA table_info(`{table_name}`)")
    columns = [row[1] for row in cursor.fetchall()]
    if 'Safety Stock impact' in columns:
            print(f"Column 'Safety Stock impact' already exists in table '{table_name}'.")
    else:
            # Add the 'Safety Stock impact' column (REAL for potential decimal values)
            cursor.execute(f"ALTER TABLE `{table_name}` ADD COLUMN `Safety Stock impact` REAL")
            print(f"Added 'Safety Stock impact' column to table '{table_name}'.")

    

    
    update_query = f"""
        UPDATE `{table_name}`
        SET `Safety Stock impact` = ROUND(`Part standard price` * `Safety Stock`,0)
    """
    cursor.execute(update_query)
    conn.commit()  
    conn.close()




add_stock_value_column(db_file, table_name)

Added 'Safety Stock impact' column to table 'my_table'.


In [21]:
def add_stock_value_column(db_file, table_name):
    
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()


    cursor.execute(f"PRAGMA table_info(`{table_name}`)")
    columns = [row[1] for row in cursor.fetchall()]
    if 'Pending Order value' in columns:
            print(f"Column 'Pending Order value' already exists in table '{table_name}'.")
    else:
            # Add the 'Pending Order value' column (REAL for potential decimal values)
            cursor.execute(f"ALTER TABLE `{table_name}` ADD COLUMN `Pending Order value` REAL")
            print(f"Added 'Pending Order value' column to table '{table_name}'.")

    

    
    update_query = f"""
        UPDATE `{table_name}`
        SET `Pending Order value` = ROUND(`Part standard price` * `Pending orders`,0)
    """
    cursor.execute(update_query)
    conn.commit()  
    conn.close()




add_stock_value_column(db_file, table_name)

Added 'Pending Order value' column to table 'my_table'.


In [22]:
def add_stock_value_column(db_file, table_name):
    
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()


    cursor.execute(f"PRAGMA table_info(`{table_name}`)")
    columns = [row[1] for row in cursor.fetchall()]
    if 'Consumption value' in columns:
            print(f"Column 'Consumption value' already exists in table '{table_name}'.")
    else:
            # Add the 'Consumption value' column (REAL for potential decimal values)
            cursor.execute(f"ALTER TABLE `{table_name}` ADD COLUMN `Consumption value` REAL")
            print(f"Added 'Consumption value' column to table '{table_name}'.")

    

    
    update_query = f"""
        UPDATE `{table_name}`
        SET `Consumption value` = ROUND(`Part standard price` * `Yearly consumption`,0)
    """
    cursor.execute(update_query)
    conn.commit()  
    conn.close()




add_stock_value_column(db_file, table_name)

Added 'Consumption value' column to table 'my_table'.


In [23]:
def add_stock_value_column(db_file, table_name):
    
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()


    cursor.execute(f"PRAGMA table_info(`{table_name}`)")
    columns = [row[1] for row in cursor.fetchall()]
    if 'Coverage class' in columns:
            print(f"Column 'Coverage class' already exists in table '{table_name}'.")
    else:
            # Add the 'Coverage class' column (REAL for potential decimal values)
            cursor.execute(f"ALTER TABLE `{table_name}` ADD COLUMN `Coverage class` TEXT")
            print(f"Added 'Coverage class' column to table '{table_name}'.")



    update_query = f"""
            UPDATE `{table_name}`
            SET `Coverage class` =
                CASE
                    WHEN `Order specific part` = 'Order specific' THEN `Order specific part`
                    WHEN `Days` IS NULL OR `Days` = '' THEN 'Not moving'
                    WHEN `Days` < 15 THEN '<15 days'
                    WHEN `Days` < 30 THEN '<30 days'
                    WHEN `Days` < 90 THEN '<90 days'
                    ELSE '>90 days'
                END
        """
    cursor.execute(update_query)
    conn.commit()  
    conn.close()




add_stock_value_column(db_file, table_name)

Added 'Coverage class' column to table 'my_table'.


In [24]:
def add_Consumption_class_column(db_file, table_name):
    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()


        cursor.execute(f"PRAGMA table_info(`{table_name}`)")
        columns = [row[1] for row in cursor.fetchall()]
        if 'Consumption class' in columns:
            print(f"Column 'Consumption class' already exists in table '{table_name}'.")
        else:
 
            cursor.execute(f"ALTER TABLE `{table_name}` ADD COLUMN `Consumption class` INTEGER")
            print(f"Added 'Consumption class' column to table '{table_name}'.")

        # Calculate percentiles using a different method
        # First, get all consumption values in sorted order
        cursor.execute(f"""
            SELECT `Consumption value`
            FROM `{table_name}`
            WHERE `Consumption value` IS NOT NULL
            ORDER BY `Consumption value`
        """)
        values = [row[0] for row in cursor.fetchall()]
        
        if values:
            # Calculate 90th and 60th percentiles
            percentile_90_idx = int(len(values) * 0.90)
            percentile_60_idx = int(len(values) * 0.60)
            percentile_90 = values[percentile_90_idx]
            percentile_60 = values[percentile_60_idx]
        else:
            percentile_90 = 0
            percentile_60 = 0

        # Update the Consumption class
        update_query = f"""
            UPDATE `{table_name}`
            SET `Consumption class` =
                CASE
                    WHEN `Order specific part` = 'Order specific' THEN 'Order specific'
                    WHEN `Consumption value` IS NULL OR `Consumption value` = 0 THEN 4
                    WHEN `Consumption value` > {percentile_90} THEN 1
                    WHEN `Consumption value` > {percentile_60} THEN 2
                    ELSE 3
                END
        """

        cursor.execute(update_query)
        conn.commit()
        print(f"Calculated and updated 'Consumption class' for all rows in table '{table_name}'.")

        conn.close()

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")




add_Consumption_class_column(db_file, table_name)

Added 'Consumption class' column to table 'my_table'.
Calculated and updated 'Consumption class' for all rows in table 'my_table'.


In [25]:
import sqlite3

def add_stock_class_column(db_file, table_name):

    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Check if 'Stock Class' column already exists
        cursor.execute(f"PRAGMA table_info(`{table_name}`)")
        columns = [row[1] for row in cursor.fetchall()]
        if 'Stock Class' in columns:
            print(f"Column 'Stock Class' already exists in table '{table_name}'.")
        else:
            # Add the 'Stock Class' column (TEXT for string values: A, B, C, D)
            cursor.execute(f"ALTER TABLE `{table_name}` ADD COLUMN `Stock Class` TEXT")
            print(f"Added 'Stock Class' column to table '{table_name}'.")

        # Calculate percentiles using the index-based method
        cursor.execute(f"""
            SELECT `Stock Value`
            FROM `{table_name}`
            WHERE `Stock Value` IS NOT NULL
            ORDER BY `Stock Value`
        """)
        values = [row[0] for row in cursor.fetchall()]

        if values:
            # Calculate 90th and 60th percentiles indices
            percentile_90_idx = int(len(values) * 0.90)
            percentile_60_idx = int(len(values) * 0.60)

            # Handle edge cases if the indices fall at the very end
            percentile_90_idx = min(percentile_90_idx, len(values) - 1) # Ensure index is within range
            percentile_60_idx = min(percentile_60_idx, len(values) - 1) # Ensure index is within range

            percentile_90 = values[percentile_90_idx]
            percentile_60 = values[percentile_60_idx]
        else:
            percentile_90 = 0  # Set to 0 if there's no data
            percentile_60 = 0  # Set to 0 if there's no data

        # Update the 'Stock Class' column
        update_query = f"""
            UPDATE `{table_name}`
            SET `Stock Class` =
                CASE
                    WHEN `Stock Value` IS NULL OR `Stock Value` = 0 THEN 'D'
                    WHEN `Stock Value` > {percentile_90} THEN 'A'
                    WHEN `Stock Value` > {percentile_60} THEN 'B'
                    ELSE 'C'
                END
        """

        cursor.execute(update_query)
        conn.commit()
        print(f"Calculated and updated 'Stock Class' for all rows in table '{table_name}'.")

        conn.close()

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")



add_stock_class_column(db_file, table_name)

Added 'Stock Class' column to table 'my_table'.
Calculated and updated 'Stock Class' for all rows in table 'my_table'.


In [26]:
import sqlite3

def add_days_objective_column(db_file, table_name):

    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        cursor.execute(f"PRAGMA table_info(`{table_name}`)")
        columns = [row[1] for row in cursor.fetchall()]
        if 'Days Objective' in columns:
            print(f"Column 'Days Objective' already exists in table '{table_name}'.")
        else:

            cursor.execute(f"ALTER TABLE `{table_name}` ADD COLUMN `Days Objective` REAL") 
            print(f"Added 'Days Objective' column to table '{table_name}'.")

        # Update the 'Days Objective' column
        update_query = f"""
            UPDATE `{table_name}`
            SET `Days Objective` =
                CASE
                    WHEN `Order specific part` = 'Order specific' THEN NULL
                    WHEN `Consumption Class` = 4 THEN 0
                    ELSE (`Consumption Class` * 7) + (CASE WHEN `Location` = 'Imported' THEN 14 ELSE 0 END)
                END
        """

        cursor.execute(update_query)
        conn.commit()
        print(f"Calculated and updated 'Days Objective' for all rows in table '{table_name}'.")

        conn.close()

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")




add_days_objective_column(db_file, table_name)

Added 'Days Objective' column to table 'my_table'.
Calculated and updated 'Days Objective' for all rows in table 'my_table'.


In [30]:
import sqlite3

def add_benefit_column(db_file, table_name):

    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Check if 'Benefit' column already exists
        cursor.execute(f"PRAGMA table_info(`{table_name}`)")
        columns = [row[1] for row in cursor.fetchall()]
        if 'Benefit' in columns:
            print(f"Column 'Benefit' already exists in table '{table_name}'.")
        else:
            # Add the 'Benefit' column (REAL for decimal values)
            cursor.execute(f"ALTER TABLE `{table_name}` ADD COLUMN `Benefit` REAL")
            print(f"Added 'Benefit' column to table '{table_name}'.")

        # Update the 'Benefit' column
        update_query = f"""
            UPDATE `{table_name}`
            SET `Benefit` =
                CASE
                    WHEN `Days Objective` IS NULL OR `Days Objective` = '' THEN 0
                    WHEN `Days Objective` = 0 THEN round(`Stock Value` * 0.66, 0)  
                    ELSE round(
                        (
                            `Stock Value` -
                            CASE
                                WHEN `Days` > `Days Objective` THEN round((`Stock Value` / CAST(`Days` AS REAL)) * `Days Objective`, 0)
                                ELSE 0
                            END
                        ) * 0.66, 0) 
                END
        """

        cursor.execute(update_query)
        conn.commit()
        print(f"Calculated and updated 'Benefit' for all rows in table '{table_name}'.")

        conn.close()

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")



add_benefit_column(db_file, table_name)

Column 'Benefit' already exists in table 'my_table'.
Calculated and updated 'Benefit' for all rows in table 'my_table'.
